<a href="https://colab.research.google.com/github/laskari/END-Program/blob/main/Assignment_4_END.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT =  data.Field(tokenize='spacy', include_lengths= True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))
print(vars(train_data.examples[0]))

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 24.1MB/s]


{'text': ['So', 'you', 'might', 'be', 'reading', 'some', 'of', 'the', 'comments', 'posted', 'on', 'this', 'film', ',', 'and', 'you', 'might', 'be', 'thinking', 'to', 'yourself', ',', '"', 'Huh', '.', 'There', 'were', 'sure', 'a', 'bunch', 'of', 'RAVE', 'REVIEWS', 'posted', 'on', 'December', '30', '.', '"', 'Funny', 'thing', 'is', ',', 'most', 'of', 'these', 'rave', 'reviews', 'sound', 'like', 'they', "'re", 'coming', 'from', 'the', 'same', 'person', ',', 'or', 'at', 'best', 'a', 'small', ',', 'coordinated', 'group', 'of', '"', 'Open', 'House', '"', 'groupies', '.', 'The', 'truth', ',', 'my', 'friends', ',', 'is', 'that', 'this', 'film', 'is', 'truly', 'unwatchable', '.', 'Just', 'because', 'it', "'s", '"', 'independent', '"', 'does', "n't", 'mean', 'it', 'gets', 'a', 'free', 'pass', '.', 'If', 'you', "'re", 'going', 'to', 'make', 'a', 'musical', ',', 'whether', 'on', 'film', 'or', 'on', 'stage', ',', 'whether', 'on', 'Broadway', 'or', 'at', 'the', 'local', 'community', 'playhouse', ','

In [3]:
for i in range(len(train_data)):
  vars(train_data.examples[i]).get('text').reverse()

print(vars(train_data.examples[0]))

for i in range(len(valid_data)):
  vars(valid_data.examples[i]).get('text').reverse()

print(vars(valid_data.examples[0]))

{'text': ['.', 'involved', 'everyone', 'for', 'embarrassed', 'deeply', 'felt', 'actually', 'I', 'where', 'experiences', 'going', '-', 'movie', 'unfortunate', 'those', 'of', 'one', "'s", 'It', '.', 'one', 'this', 'save', "n't", 'ca', 'Rapp', 'Anthony', 'Even', '.', 'charming', 'and', 'witty', 'are', 'that', 'lyrics', 'have', 'you', ')', 'e.', '(', 'and', ',', 'score', 'written', '-', 'well', 'a', 'have', 'you', ')', 'd.', '(', ',', 'equipment', 'sound', 'decent', 'have', 'you', ')', 'c.', '(', ',', 'dance', 'can', 'actors', 'your', ')', 'b.', '(', ',', 'sing', 'can', 'actors', 'your', ')', 'a.', '(', 'that', 'sure', 'make', 'probably', 'should', 'you', ',', 'playhouse', 'community', 'local', 'the', 'at', 'or', 'Broadway', 'on', 'whether', ',', 'stage', 'on', 'or', 'film', 'on', 'whether', ',', 'musical', 'a', 'make', 'to', 'going', "'re", 'you', 'If', '.', 'pass', 'free', 'a', 'gets', 'it', 'mean', "n't", 'does', '"', 'independent', '"', "'s", 'it', 'because', 'Just', '.', 'unwatchable'

In [4]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = 'glove.6B.100d', unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                          
100%|█████████▉| 399325/400000 [00:17<00:00, 22693.58it/s]

In [5]:
BATCH_SIZE =64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size = BATCH_SIZE, sort_within_batch = True, device = device)

In [6]:
class RNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx =pad_idx)

    self.rnn_1 = nn.LSTM(embedding_dim, hidden_dim, num_layers =n_layers, bidirectional=bidirectional, dropout = dropout)
    self.rnn_2 = nn.LSTM(hidden_dim*2, hidden_dim, num_layers =n_layers, bidirectional=bidirectional, dropout = dropout)
    
    self.fc = nn.Linear(hidden_dim*2, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, text, text_lengths):
    
    embedded = self.dropout(self.embedding(text))

    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)

    packed_output, (hidden, cell) = self.rnn_1(packed_embedded)

    for i  in range(1,3):
      packed_output, (hidden, cell) = self.rnn_2(packed_output)
    
    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

    hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]),dim=1))
    return self.fc(hidden)


In [7]:
INPUT_DIM =  len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM =1
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [8]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [9]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [10]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

In [11]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


In [12]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [13]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

100%|█████████▉| 399325/400000 [00:29<00:00, 22693.58it/s]

In [14]:
def binary_accuracy(preds,y):
  rounded_preds = torch.round(torch.sigmoid(preds))

  correct = (rounded_preds == y).float()

  acc = correct.sum() / len(correct)
  return acc

In [15]:
def train(model, iterator, optimizer, criterion):
  epoch_loss =0
  epoch_acc =0

  model.train()

  for batch in iterator:

    optimizer.zero_grad()

    text, text_lengths = batch.text

    text_lengths = text_lengths.cpu()

    predictions =  model(text, text_lengths).squeeze(1)

    loss = criterion(predictions, batch.label)

    acc = binary_accuracy(predictions, batch.label)

    loss.backward()

    optimizer.step()

    epoch_acc += acc.item()
    epoch_loss += loss.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            text_lengths = text_lengths.cpu()
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [18]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 50s
	Train Loss: 0.612 | Train Acc: 66.18%
	 Val. Loss: 0.602 |  Val. Acc: 64.96%
Epoch: 02 | Epoch Time: 0m 53s
	Train Loss: 0.499 | Train Acc: 75.60%
	 Val. Loss: 0.543 |  Val. Acc: 75.64%
Epoch: 03 | Epoch Time: 0m 55s
	Train Loss: 0.341 | Train Acc: 85.57%
	 Val. Loss: 0.312 |  Val. Acc: 87.39%
Epoch: 04 | Epoch Time: 0m 55s
	Train Loss: 0.256 | Train Acc: 90.15%
	 Val. Loss: 0.288 |  Val. Acc: 88.16%
Epoch: 05 | Epoch Time: 0m 55s
	Train Loss: 0.208 | Train Acc: 92.18%
	 Val. Loss: 0.283 |  Val. Acc: 89.11%


In [19]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.345 | Test Acc: 86.03%


In [20]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [21]:
predict_sentiment(model, "This film is terrible")

0.037020716816186905

In [30]:
predict_sentiment(model, "This film is Superb")

0.9683725833892822